In [ ]:
#default_exp s3

# S3Cache

## Deals with everything that reads and write to the s3 cache for the database

In [ ]:
#hide
import pickle, os

os.environ['DATABASE_TABLE_NAME'] = 'product-table-dev-manual'
os.environ['REGION'] = 'ap-southeast-1'
os.environ['INVENTORY_BUCKET_NAME'] = 'product-bucket-dev-manual'
os.environ['INPUT_BUCKET_NAME'] = 'input-product-bucket-dev-manual'
os.environ['DAX_ENDPOINT'] = 'longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111'
os.environ['LINEKEY'] = '2uAfV4AoYglUGmKTAk2xNOm0aV2Ufgh1BQPvQl9vJd4'
REGION = 'ap-southeast-1'

In [ ]:
#export
from s3bz.s3bz import S3
from nicHelper.wrappers import add_method, add_class_method, add_static_method
from nicHelper.dictUtil import stripDict, printDict, hashDict, saveStringToFile, loadStringFromFile, saveDictToFile, loadDictFromFile
from nicHelper.exception import errorString
import nicHelper.pdUtils as pdUtils
from dict_hash import dict_hash, sha256
from base64 import b64encode, b64decode
from pandas.util import hash_pandas_object
from hashlib import sha1
import pandas as pd
import os, logging, zlib

In [ ]:
from villaProductDatabase.database import ProductDatabase
import logging
logging.basicConfig(level=logging.INFO)

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


In [ ]:
#export
import os
DBHASHLOCATION = '/mnt/efs/database.hash'
DBCACHELOCATION = '/mnt/efs/database.cache'
DATABASE_TABLE_NAME = os.environ.get('DATABASE_TABLE_NAME')
INVENTORY_BUCKET_NAME = os.environ.get('INVENTORY_BUCKET_NAME')
INPUT_BUCKET_NAME = os.environ.get('INPUT_BUCKET_NAME')
REGION = os.environ.get('REGION') or 'ap-southeast-1'
ACCESS_KEY_ID = os.environ.get('USER') or None
SECRET_ACCESS_KEY = os.environ.get('PW') or None
LINEKEY= os.environ.get('LINEKEY')
ALLDATAKEY = 'allData'
DEFAULTKEYS = ['cprcode', 'iprcode', 'oprcode', 'ordertype', 'pr_abb', 'pr_active', 'pr_cgcode', 'pr_code', 'pr_dpcode', 'pr_engname', 'pr_ggcode', 'pr_market', 'pr_name', 'pr_puqty', 'pr_sa_method', 'pr_sucode1', 'pr_suref3', 'prtype', 'psqty', 'pstype']
  
try:
  DAX_ENDPOINT = os.environ['DAX_ENDPOINT']
  print(DAX_ENDPOINT)
except KeyError as e:
  print(f'dax endpoint missing {e}')
  

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


In [ ]:
#export
class S3Cache:
  pass

In [ ]:
class Tester( S3Cache, ProductDatabase):
  class Meta:
    table_name = os.environ['DATABASE_TABLE_NAME']
    region = os.environ['REGION']
    billing_mode='PAY_PER_REQUEST'
    dax_read_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
    dax_write_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
  pass
  

## save and load local hash

In [ ]:
#export
@add_class_method(S3Cache)
def loadFromCache(cls, key=ALLDATAKEY, localCache=DBCACHELOCATION, 
                  localHash=DBHASHLOCATION, bucket=INVENTORY_BUCKET_NAME):
  ## check for local object and its hash
  if os.path.exists(localCache) and os.path.exists(localHash):
    try:
      localHash = pdUtils.loadLocalHash(path=localHash)
      logging.debug(f'localHash is {localHash}')
      remoteHash = pdUtils.loadRemoteHash(key=key, bucket=bucket)
      logging.debug(f'remoteHash is {remoteHash}')

      if localHash == remoteHash:
        print('data is still in sync, using local cache')
        db = pdUtils.loadLocalCache(path=localCache)
        return db
      else: 
        print('remote hash is not the same, load remote cache')
    except Exception as e: print(f'local loading error{e}, loading remote hash')
  ### load from remote cache
  try:
    db = pdUtils.loadRemoteCache(key=key, bucket=bucket)
    pdUtils.saveLocalCache(db, path=localCache)
    pdUtils.saveLocalHash(db, path = localHash)
  except Exception as e:
    print(f'locding remtoe failed {e} returning blank df')
    db = pd.DataFrame(columns = DEFAULTKEYS)
  ### save to local cache
  return db

In [ ]:
sampleDict = {'cprcode': '0171670', 'iprcode': '0171670', 'oprcode': '0171670', 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAPAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845     ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}
sampleDict.keys()
keys = ['cprcode', 'iprcode', 'oprcode', 'ordertype', 'pr_abb', 'pr_active', 'pr_cgcode', 'pr_code', 'pr_dpcode', 'pr_engname', 'pr_ggcode', 'pr_market', 'pr_name', 'pr_puqty', 'pr_sa_method', 'pr_sucode1', 'pr_suref3', 'prtype', 'psqty', 'pstype']
pd.DataFrame(columns = keys)
DUMMYDATA = {'cprcode': '000000', 'iprcode': '000000', 'oprcode': '0171670', 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAPAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845     ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}

In [ ]:
%%time
db = Tester.loadFromCache()
assert db.shape > (10,10)

INFO:root:using accelerate endpoint
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


loading hashkey allData-hash


INFO:root:using accelerate endpoint


loaded hash is d0c0857f93ea21a99bc59a7c459dea6b8341d7cf
data is still in sync, using local cache


AssertionError: 

In [ ]:
Tester.loadFromCache(key='doestExist', localCache='doesntExist', localHash='doesntExist')

INFO:root:using accelerate endpoint


locding remtoe failed An error occurred (404) when calling the HeadObject operation: Not Found returning blank df


,cprcode,iprcode,oprcode,ordertype,pr_abb,pr_active,pr_cgcode,pr_code,pr_dpcode,pr_engname,pr_ggcode,pr_market,pr_name,pr_puqty,pr_sa_method,pr_sucode1,pr_suref3,prtype,psqty,pstype


## Save and load remote hash

In [ ]:
#export

@add_class_method(S3Cache)
def saveRemoteCache(cls ,db:pd.DataFrame, key= ALLDATAKEY,
                   bucket = INVENTORY_BUCKET_NAME, localCachePath=DBCACHELOCATION,
                   localHashPath=DBHASHLOCATION):
  if db.empty:
    db = pd.DataFrame([DUMMYDATA])
  db.columns = db.columns.astype(str)
  db = db.reset_index(drop=True)
  
  print(db)
  pdUtils.saveRemoteCache(data=db, key= key, 
                          bucket=bucket, localCachePath=localCachePath, 
                          localHashPath=localHashPath)
  jsonDb = db.to_json(orient='split')
  zlibArc = zlib.compress(jsonDb.encode())
  tmpPath = '/tmp/zlibJsonCache.zl'
  with open(tmpPath, 'wb') as f:
    f.write(zlibArc)
  S3.saveFile(key=f'{key}-json.zl',path=tmpPath,bucket=bucket)
  


In [ ]:
%%time
db = pd.DataFrame([sampleDict])
Tester.saveRemoteCache(db)
assert pdUtils.getDfHash(db.reset_index(drop=True)) == pdUtils.getDfHash(Tester.loadFromCache()), 'save fail'

INFO:root:using accelerate endpoint
INFO:root:data was saved to s3
INFO:root:using accelerate endpoint


   cprcode  iprcode  oprcode ordertype               pr_abb pr_active  \
0  0171670  0171670  0171670         Y  JIRAPAT YOUG KALE 2         Y   

  pr_cgcode  pr_code pr_dpcode                 pr_engname pr_ggcode  \
0        05  0171670        19  JIRAPAT YOUNG KALE 200 G.       057   

                 pr_market                  pr_name pr_puqty pr_sa_method  \
0  JIRAPAT ยอดคะน้า 200 G.  JIRAPAT ยอดคะน้า 200 G.        1            1   

   pr_sucode1 pr_suref3 prtype psqty pstype  
0  CM845              A      I     1      1  
hashKey is allData-hash
saving hash to s3
saved hash d0c0857f93ea21a99bc59a7c459dea6b8341d7cf


INFO:root:using accelerate endpoint
INFO:root:using accelerate endpoint
INFO:root:using accelerate endpoint


loading hashkey allData-hash
loaded hash is d0c0857f93ea21a99bc59a7c459dea6b8341d7cf
data is still in sync, using local cache
CPU times: user 123 ms, sys: 11.7 ms, total: 134 ms
Wall time: 529 ms


In [ ]:
Tester.loadFromCache()

INFO:root:using accelerate endpoint
INFO:root:using accelerate endpoint


loading hashkey allData-hash
loaded hash is d0c0857f93ea21a99bc59a7c459dea6b8341d7cf
data is still in sync, using local cache


,cprcode,iprcode,oprcode,ordertype,pr_abb,pr_active,pr_cgcode,pr_code,pr_dpcode,pr_engname,pr_ggcode,pr_market,pr_name,pr_puqty,pr_sa_method,pr_sucode1,pr_suref3,prtype,psqty,pstype
0,0171670,0171670,0171670,Y,JIRAPAT YOUG KALE 2,Y,05,0171670,19,JIRAPAT YOUNG KALE 200 G.,057,JIRAPAT ยอดคะน้า 200 G.,JIRAPAT ยอดคะน้า 200 G.,1,1,CM845,A,I,1,1


In [ ]:
db.reset_index(drop=True).to_feather('/tmp/test')

## Reset S3 Cache

In [ ]:
#export
@add_class_method(S3Cache)
def resetS3Cache(cls, bucketName= INVENTORY_BUCKET_NAME, key = 'allData', limit=10000, **kwargs):
  ''' upload changes to s3'''
  ###### get all data
  items:List[cls] = cls.scanDb(limit=limit)
  db:pd.DataFrame = cls.toDf(items)
  print(f'{db.shape} changes to update')
  cls.saveRemoteCache(db)
  return True

In [ ]:
%%time
Tester.resetS3Cache()

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


(1, 20) changes to update
   cprcode  iprcode  oprcode ordertype                pr_abb pr_active  \
0  0171670  0171670  0171670         Y  JIRAPAT YOUNG KALE 2         Y   

  pr_cgcode  pr_code pr_dpcode                pr_engname pr_ggcode  \
0        05  0171670        19  JIRAAT YOUNG KALE 200 G.       057   

                 pr_market                  pr_name pr_puqty pr_sa_method  \
0  JIRAPAT ยอดคะน้า 200 G.  JIRAPAT ยอดคะน้า 200 G.        1            1   

  pr_sucode1 pr_suref3 prtype psqty pstype  
0      CM845         A      I     1      1  


INFO:root:using accelerate endpoint
INFO:root:data was saved to s3
INFO:root:using accelerate endpoint


hashKey is allData-hash
saving hash to s3
saved hash 43a007725a68e2be2ca47be21c49845cf840b3ef


INFO:root:using accelerate endpoint


CPU times: user 139 ms, sys: 3.67 ms, total: 142 ms
Wall time: 523 ms


True

In [ ]:
jsonDb = db.to_json(orient='split')

In [ ]:
import zlib, json
%time jsonDb = db.to_json(orient='split')
%time zlibArc = zlib.compress(jsonDb.encode())
%time zlibObject = zlib.decompress(zlibArc).decode()
%time dbDict = json.loads(zlibObject)

CPU times: user 100 µs, sys: 18 µs, total: 118 µs
Wall time: 124 µs
CPU times: user 49 µs, sys: 9 µs, total: 58 µs
Wall time: 62.7 µs
CPU times: user 376 µs, sys: 70 µs, total: 446 µs
Wall time: 312 µs
CPU times: user 26 µs, sys: 4 µs, total: 30 µs
Wall time: 35.5 µs


In [ ]:
import sys
print(sys.getsizeof(jsonDb))
print(sys.getsizeof(zlibArc))

589
279
